In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, GridSearchCV
from sklearn.feature_extraction import DictVectorizer as DV
from sklearn.preprocessing import OrdinalEncoder

from sklearn.metrics import roc_auc_score
from sklearn.metrics import *

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from matplotlib import pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter('ignore', category=FutureWarning)

import warnings 
warnings.filterwarnings("ignore") 

Открываем данные:

In [2]:
data = pd.read_csv('train_data.csv')
labels = pd.read_csv('train_labels.csv')

Добавим функцию по удалению стоблцов в которых все строки пустые

In [3]:
def get_nan_cols(df, nan_percent=0.8):
    threshold = len(df.index) * nan_percent
    return [c for c in df.columns if sum(df[c].isnull()) >= threshold]

Отделяем столбцы с количественными и категориальными переменными согласно условиям задания

In [4]:
numeric_cols = data.columns.values[:190]
categorical_cols = list(set(data.columns.values.tolist()) - set(numeric_cols))

Запускаем функцию по удалению стоблцов в которых все строки пустые

In [5]:
X_real = data[numeric_cols].drop(data[numeric_cols][get_nan_cols(data[numeric_cols], 1)].columns, axis = 1)
X_cat = data[categorical_cols].drop(data[categorical_cols][get_nan_cols(data[categorical_cols], 0.8)].columns, axis = 1)

Работаем с пустыми строками. Количественные признаки будет рассматривать с заменой на '0' и на среднее значение по столбцу. Категориальные - делаем замену на 'NA'.

In [6]:
X_real_zeros = X_real.fillna(0)
X_real_mean = X_real.fillna(data.mean(axis=0))
X_cat = X_cat.fillna('NA', axis=0).applymap(str)

Преобразовываем категориальные признаки с помощью Ordinal encoding

In [7]:
def OrdinalEnc(data, cat_features):
    data1 = data.copy()
    enc = OrdinalEncoder()
    data1[cat_features] = enc.fit_transform(data[cat_features].astype(str))
    return data1

In [8]:
enc_data = OrdinalEnc(X_cat,X_cat.columns)

In [9]:
X = pd.concat((X_real_mean, enc_data), axis = 1)

Строим модели.
Замечание: Далее будет использоваться количественные признаки с заменой пустых на среднее, так как модели дали эффективность выше на 5-7 пунктов. Этап сравнения был опущен, так как главная задача задания построение базовых моделей. 

1. Logistic Regression Baseline:

In [48]:
logit = LogisticRegression()

In [49]:
%%time
cv_scores_lr1 = cross_val_score(logit, 
                             X, 
                             labels, 
                             cv = 5, 
                            scoring = 'roc_auc')

CPU times: user 2min 32s, sys: 847 ms, total: 2min 33s
Wall time: 2min 33s


In [51]:
%%time
cv_scores_lr2 = cross_val_score(logit, 
                             X, 
                             labels, 
                             cv = 5, 
                            scoring='f1')

CPU times: user 2min 41s, sys: 1.1 s, total: 2min 42s
Wall time: 2min 42s


In [50]:
print ('Результаты основной метрики ROC_AUC:',cv_scores_lr1, cv_scores_lr1.mean())

Результаты основной метрики ROC_AUC: [0.58525124 0.59749238 0.57900783 0.60056082 0.57266498] 0.5869954509311632


In [52]:
print ('Результаты вспомогательной метрики F1:', cv_scores_lr2, cv_scores_lr2.mean())

Результаты вспомогательной метрики F1: [0.         0.00478469 0.         0.         0.00472813] 0.0019025642765844336


2. Random Forest Baseline:

In [53]:
rf = RandomForestClassifier()

In [54]:
%%time

cv_scores_rf1 = cross_val_score(rf, 
                             X, 
                             labels, 
                             cv = 5, 
                            scoring = 'roc_auc')

CPU times: user 5.17 s, sys: 271 ms, total: 5.45 s
Wall time: 5.52 s


In [55]:
%%time

cv_scores_rf2 = cross_val_score(rf, 
                             X, 
                             labels, 
                             cv = 5, 
                            scoring = 'f1')

CPU times: user 5.01 s, sys: 286 ms, total: 5.3 s
Wall time: 5.34 s


In [56]:
print ('Результаты основной метрики ROC_AUC:', cv_scores_rf1, cv_scores_rf1.mean())

Результаты основной метрики ROC_AUC: [0.59566629 0.60844588 0.60149395 0.61960909 0.60014967] 0.6050729760630609


In [57]:
print ('Результаты вспомогательной метрики F1:', cv_scores_rf2, cv_scores_rf2.mean())

Результаты вспомогательной метрики F1: [0.00947867 0.00478469 0.00945626 0.00952381 0.01408451] 0.009465588664494812


3. Gradient Boosting:

In [11]:
gbc = GradientBoostingClassifier()

In [59]:
%%time

cv_scores_gbc1 = cross_val_score(gbc, 
                             X, 
                             labels, 
                             cv = 5, 
                            scoring = 'roc_auc')

CPU times: user 1min 22s, sys: 623 ms, total: 1min 22s
Wall time: 1min 23s


In [60]:
%%time

cv_scores_gbc2 = cross_val_score(gbc, 
                             X, 
                             labels, 
                             cv = 5, 
                            scoring = 'f1')

CPU times: user 1min 20s, sys: 429 ms, total: 1min 21s
Wall time: 1min 21s


In [61]:
print ('Результаты основной метрики ROC_AUC:', cv_scores_gbc1, cv_scores_gbc1.mean())

Результаты основной метрики ROC_AUC: [0.72703296 0.71317899 0.74408519 0.74104923 0.7217744 ] 0.7294241547394483


In [62]:
print ('Результаты основной метрики F1:', cv_scores_gbc2, cv_scores_gbc2.mean())

Результаты основной метрики F1: [0.02314815 0.0141844  0.02790698 0.00475059 0.01864802] 0.01772762690554029


Kaggle:

In [10]:
test_data = pd.read_csv('orange_small_churn_test_data.csv')

In [11]:
X_real = test_data[numeric_cols].drop(test_data[numeric_cols][get_nan_cols(test_data[numeric_cols], 1)].columns, axis = 1)
X_cat = test_data[categorical_cols].drop(test_data[categorical_cols][get_nan_cols(test_data[categorical_cols], 0.8)].columns, axis = 1)

X_real_zeros = X_real.fillna(0)
X_real_mean = X_real.fillna(data.mean(axis=0))
X_cat = X_cat.fillna('NA', axis=0).applymap(str)

enc_data = OrdinalEnc(X_cat,X_cat.columns)

X_test = pd.concat((X_real_mean, enc_data), axis = 1)

In [12]:
def make_csv_predictions(predictions, path = None):
    df = pd.DataFrame({'ID': range(0, len(predictions)), 'result': predictions})
    if path is not None:
        df.to_csv(path, sep = ',', index = False)
    else:
        return df

In [13]:
gbc = GradientBoostingClassifier().fit(X, labels)

# Make a prediction for test data set

y_test = gbc.predict_proba(X_test)[:, 1]

# Write it to the file which could be submitted


In [14]:
make_csv_predictions(y_test, path = 'naive_logistic.csv')

Как мы видим, за baseline лучше взять градиентный бустин - он дает наиболее высокий скор и удовлетворительное время работы. Конечно, не так быстро как случайный лес, но время все равно допустимое даже с учетом дальнейшего тюнинга параметров модели.